# Module 2 Assessment

Welcome to your Mod 2 Assessment. You will be tested for your understanding of concepts and ability to solve problems that have been covered in class and in the curriculum.

Use any libraries you want to solve the problems in the assessment.

You will have up to two hours to complete this assessment.

The sections of the assessment are:

- Accessing Data Through APIs
- Object Oriented Programming
- SQL and Relational Databases
- HTML, CSS and Web Scraping
- Other Database Structures (MongoDB)

In this assessment you will be exploring two datasets: Pokemon and Quotes.

In [2]:
# import the necessary libraries

import requests
import json
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup

## Part 1: Accessing Data Through APIs

In this section we'll be using PokeAPI to get data on Pokemon. Let's first define functions to get information from the API. Provided below is a URL that will get you started with the first 151 Pokemon! Run the cell below to see what we get.

In [5]:
url = 'https://pokeapi.co/api/v2/pokemon/?limit=151'
results = requests.get(url).json()['results']
results

[{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
 {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
 {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
 {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
 {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
 {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
 {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
 {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
 {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
 {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
 {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
 {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
 {'name': 'weedle', 'url': 'https://pokeapi.co/api/v2/pokemon/13/'},
 {'name': 'kakuna', 'url': 'https://pokeapi.co/api/v2/pokemon/14/'},
 {'name': '

[Read the documentation here](https://pokeapi.co/) for information on navigating this API and use the API to obtain data to answer the following questions.

### Accessing Data

1. For any **one** Pokemon, retrieve the following information in a dictionary format with the following keys:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities

For `Types` and `Abilities`, you might want to write helper functions to have each attribute just be a list of types and a list of abilities. Your output should look like this:

```
{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}

```
    


In [14]:
poke1 = requests.get(results[0]['url']).json()

In [17]:
poke1

{'abilities': [{'ability': {'name': 'chlorophyll',
    'url': 'https://pokeapi.co/api/v2/ability/34/'},
   'is_hidden': True,
   'slot': 3},
  {'ability': {'name': 'overgrow',
    'url': 'https://pokeapi.co/api/v2/ability/65/'},
   'is_hidden': False,
   'slot': 1}],
 'base_experience': 64,
 'forms': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon-form/1/'}],
 'game_indices': [{'game_index': 1,
   'version': {'name': 'white-2',
    'url': 'https://pokeapi.co/api/v2/version/22/'}},
  {'game_index': 1,
   'version': {'name': 'black-2',
    'url': 'https://pokeapi.co/api/v2/version/21/'}},
  {'game_index': 1,
   'version': {'name': 'white',
    'url': 'https://pokeapi.co/api/v2/version/18/'}},
  {'game_index': 1,
   'version': {'name': 'black',
    'url': 'https://pokeapi.co/api/v2/version/17/'}},
  {'game_index': 1,
   'version': {'name': 'soulsilver',
    'url': 'https://pokeapi.co/api/v2/version/16/'}},
  {'game_index': 1,
   'version': {'name': 'heartgold',
    'ur

In [15]:
# you may define helper functions for types and abilities here
def abilities(pokemon):
    abils = []
    #loop through list of dictionaries
    for abil in pokemon['abilities']:
        abils.append(abil['ability']['name'])    #get names of abilities and store in list
    #return list
    return abils

def types(pokemon):
    types = []
    for poketype in pokemon['types']:
        types.append(poketype['type']['name'])
    return types

In [16]:
types(poke1)

['poison', 'grass']

In [18]:
def get_pokedata(pokemon):
    
    """
    url is a string of the URL that will get you the relevant info from the API
    
    this function should return the dictionary for ONE pokemon
    
    """
    poke = requests.get(pokemon['url']).json()
    poke1 = {}
    poke1['id'] = poke['id']
    poke1['name'] = pokemon['name']
    poke1['base_exp'] = poke['base_experience']
    poke1['weight'] = poke['weight']
    poke1['height'] = poke['height']
    poke1['types'] = types(poke)
    poke1['abilities'] = abilities(poke)
    
    return poke1


In [19]:
get_pokedata(results[0])

{'id': 1,
 'name': 'bulbasaur',
 'base_exp': 64,
 'weight': 69,
 'height': 7,
 'types': ['poison', 'grass'],
 'abilities': ['chlorophyll', 'overgrow']}

### Pagination

2. Get the same information for the first **151** Pokemon as a list of dictionaries ordered by Pokemon ID. Print the first and last elements of the list. (Hint: Use pagination) Your output should save the list to a variable and look like this:

```
[{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, 
{'id': 2, 
'name': 'ivysaur', 
'base_experience': 142, 
'weight': 130, 
'height': 10, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, ... ]

```



In [20]:
""" 

Assign to "pokedata" the list of 151 dictionaries.
You may use your function from the previous question.

"""

# Your code here



pokedata = [get_pokedata(result) for result in results]


In [21]:
# printing first and last elements

print(pokedata[0], pokedata[-1])

{'id': 1, 'name': 'bulbasaur', 'base_exp': 64, 'weight': 69, 'height': 7, 'types': ['poison', 'grass'], 'abilities': ['chlorophyll', 'overgrow']} {'id': 151, 'name': 'mew', 'base_exp': 270, 'weight': 40, 'height': 4, 'types': ['psychic'], 'abilities': ['synchronize']}


## Part 2: Object Oriented Programming

We're going to use the data gathered in the previous section on APIs for this section on Object Oriented Programming to instantiate Pokemon objects and write instance methods.

### Creating a Class

1. Create a class called `Pokemon` with an `__init__` method to instantiate the following attributes:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities
    



In [29]:
# if you were unable to get the data from the API in the right format,
# uncomment the code below to access a JSON file with the list of dictionaries

# with open('data/pokemon.json') as f:  
#     pokelist = json.load(f)

In [31]:
pokedata

[{'id': 1,
  'name': 'bulbasaur',
  'base_exp': 64,
  'weight': 69,
  'height': 7,
  'types': ['poison', 'grass'],
  'abilities': ['chlorophyll', 'overgrow']},
 {'id': 2,
  'name': 'ivysaur',
  'base_exp': 142,
  'weight': 130,
  'height': 10,
  'types': ['poison', 'grass'],
  'abilities': ['chlorophyll', 'overgrow']},
 {'id': 3,
  'name': 'venusaur',
  'base_exp': 236,
  'weight': 1000,
  'height': 20,
  'types': ['poison', 'grass'],
  'abilities': ['chlorophyll', 'overgrow']},
 {'id': 4,
  'name': 'charmander',
  'base_exp': 62,
  'weight': 85,
  'height': 6,
  'types': ['fire'],
  'abilities': ['solar-power', 'blaze']},
 {'id': 5,
  'name': 'charmeleon',
  'base_exp': 142,
  'weight': 190,
  'height': 11,
  'types': ['fire'],
  'abilities': ['solar-power', 'blaze']},
 {'id': 6,
  'name': 'charizard',
  'base_exp': 240,
  'weight': 905,
  'height': 17,
  'types': ['flying', 'fire'],
  'abilities': ['solar-power', 'blaze']},
 {'id': 7,
  'name': 'squirtle',
  'base_exp': 63,
  'weight

In [59]:
"""
Create your class below with the correct syntax, including an __init__ method.

"""
class Pokemon:
    def __init__(self, data):
        self.data = data
        self.ID = self.data['id']
        self.name = self.data['name']
        self.exp = self.data['base_exp']
        self.weight = self.data['weight']
        self.height = self.data['height']
        self.types = self.data['types']
        self.abilities = self.data['abilities']
    
    def bmi(self):
        height = int(self.height) * 0.1
        weight = int(self.weight) * 0.1
        return weight/(height**2)

    
### Instantiating Objects

2. Using the data you obtained from the API, instantiate the first, fourth and seventh Pokemon. Assign them to the variables `bulbasaur`, `charmander` and `squirtle`.

In [60]:
# Your code here
bulbasaur = Pokemon(pokedata[0])
charmander = Pokemon(pokedata[3])
squirtle = Pokemon(pokedata[6])

In [61]:
bulbasaur.bmi()

14.081632653061222

In [56]:
# run this cell to test and check your code
# you may need to edit the attribute variable names if you named them differently!

def print_pokeinfo(pokemon_object):
    o = pokemon_object
    print('ID: ' + str(o.ID) + '\n' +
          'Name: ' + o.name.title() + '\n' +
          'Base experience: ' + str(o.exp) + '\n' +
          'Weight: ' + str(o.weight) + '\n' +
          'Height: ' + str(o.height) + '\n' +
          'Types: ' + str(o.types) + '\n' +
          'Abilities: ' + str(o.abilities) + '\n'
         )
    
print_pokeinfo(bulbasaur)
print_pokeinfo(charmander)
print_pokeinfo(squirtle)

ID: 1
Name: Bulbasaur
Base experience: 64
Weight: 69
Height: 7
Types: ['poison', 'grass']
Abilities: ['chlorophyll', 'overgrow']

ID: 4
Name: Charmander
Base experience: 62
Weight: 85
Height: 6
Types: ['fire']
Abilities: ['solar-power', 'blaze']

ID: 7
Name: Squirtle
Base experience: 63
Weight: 90
Height: 5
Types: ['water']
Abilities: ['rain-dish', 'torrent']



### Instance Methods

3. Write an instance method within the class `Pokemon` to find the BMI of a Pokemon. BMI is defined by $\frac{weight}{height^{2}}$ with weight in **kilograms** and height in **meters**. The height and weight data of Pokemon from the API is in **decimeters** and **hectograms** respectively.


    1 decimeter = 0.1 meters
    1 hectogram = 0.1 kilograms

In [62]:
# run this cell to test and check your code
# you will probably have to rerun the code to instantiate your objects

print(bulbasaur.bmi()) # 14.08
print(charmander.bmi()) # 23.61
print(squirtle.bmi()) # 36

14.081632653061222
23.611111111111104
36.0


## Part 3: SQL and Relational Databases

For this section, we've put the Pokemon data into SQL tables. You won't need to use your list of dictionaries or the JSON file for this section. The schema of `pokemon.db` is as follows:

<img src="data/pokemondb.png" alt="db schema" style="width:500px;"/>

Assign your SQL queries as strings to the variables `q1`, `q2`, etc. and run the cells at the end of this section to print your results as Pandas DataFrames.

- q1: query all columns from `Pokemon` the Pokemon that have base_experience above 200  

  
- q2: query the id, name, type1 and type2 of Pokemon that have **water** types as either their first or second type


- q3: query the average weight of Pokemon by their first type in descending order


- q4: query the Pokemon name, Pokemon type2, and what **type2** has "2xdamage" to


- q5: query the top 5 most common type1s, the minimum height, maximum height, minimum weight and maximum weight of pokemon with those type1s, and what associated type they do "0.5xdamage" to


**Important note on syntax**: use `double quotes ""` when quoting strings **within** your query and wrap the entire query in `single quotes ''` For the column titles that begin with numbers, you need to wrap the column names in double quotes.

In [63]:
cnx = sqlite3.connect('data/pokemon.db')

In [65]:
q1 = """SELECT * FROM pokemon
        WHERE base_experience > 200"""
pd.read_sql(q1, cnx)

,id,name,base_experience,weight,height,type1,type2
0,3,venusaur,236,1000,20,grass,poison
1,6,charizard,240,905,17,fire,flying
2,9,blastoise,239,855,16,water,None
3,18,pidgeot,216,395,15,normal,flying
4,26,raichu,218,300,8,electric,None
5,31,nidoqueen,227,600,13,poison,ground
6,34,nidoking,227,620,14,poison,ground
7,36,clefable,217,400,13,fairy,None
8,45,vileplume,221,186,12,grass,poison
9,62,poliwrath,230,540,13,water,fighting


In [73]:
q2 = """SELECT id, name, type1, type2 FROM pokemon
        WHERE type1 LIKE '%water%' OR type2 LIKE '%water%'"""
pd.read_sql(q2, cnx)

,id,name,type1,type2
0,7,squirtle,water,None
1,8,wartortle,water,None
2,9,blastoise,water,None
3,54,psyduck,water,None
4,55,golduck,water,None
5,60,poliwag,water,None
6,61,poliwhirl,water,None
7,62,poliwrath,water,fighting
8,72,tentacool,water,poison
9,73,tentacruel,water,poison


In [81]:
q3 = """SELECT type1, AVG(weight) FROM pokemon
        GROUP BY type1
        ORDER BY 2 DESC"""
pd.read_sql(q3, cnx)

,type1,AVG(weight)
0,rock,876.111111
1,dragon,766.000000
2,water,579.678571
3,fighting,542.857143
4,psychic,515.625000
5,normal,500.863636
6,fire,480.250000
7,ice,480.000000
8,ground,452.625000
9,electric,317.888889


In [110]:
q4 = """SELECT p.name, p.type2, [2xdamage] FROM pokemon p
        JOIN types t ON type2 = t.name"""
pd.read_sql(q4, cnx)

,name,type2,2xdamage
0,bulbasaur,poison,grass
1,ivysaur,poison,grass
2,venusaur,poison,grass
3,charizard,flying,fighting
4,butterfree,flying,fighting
5,weedle,poison,grass
6,kakuna,poison,grass
7,beedrill,poison,grass
8,pidgey,flying,fighting
9,pidgeotto,flying,fighting


In [133]:
q5 = """SELECT type1, MIN(weight), MAX(weight), MIN(height), MAX(height), t.[0.5xdamage] FROM pokemon p
        JOIN types t on p.type1 = t.name
        GROUP BY type1
        ORDER BY COUNT(type1) DESC
        LIMIT 5"""
pd.read_sql(q5, cnx)

,type1,MIN(weight),MAX(weight),MIN(height),MAX(height),0.5xdamage
0,water,40,2350,3,65,steel
1,normal,18,4600,3,22,None
2,poison,10,650,4,35,fighting
3,bug,29,560,3,15,fighting
4,fire,85,1550,6,20,bug


## Section 4: Web Scraping

### Accessing Data Using BeautifulSoup

Use BeautifulSoup to get quotes, authors, and tags from [Quotes to Read](http://quotes.toscrape.com/).

Before answering these questions, go to the site and inspect the page. Make sure to look at what links there are and how the site is structured.

1. Get the first author and the path for the author's page as a tuple from the [homepage](http://quotes.toscrape.com/).

In [225]:
# Make a get request to retrieve the page
html_page = requests.get('http://quotes.toscrape.com/') 
# Pass the page contents to beautiful soup for parsing
soup = BeautifulSoup(html_page.content, 'html.parser')

# Your code here
def get_author(some_soup):
    auth = soup.find_all('small', class_ = 'author')
    return (auth[0].text, auth[0].next_sibling.next_sibling['href'])

get_author(soup)

('Albert Einstein', '/author/Albert-Einstein')

In [227]:
soup.find_all(class_ = 'author')

[<small class="author" itemprop="author">Albert Einstein</small>,
 <small class="author" itemprop="author">J.K. Rowling</small>,
 <small class="author" itemprop="author">Albert Einstein</small>,
 <small class="author" itemprop="author">Jane Austen</small>,
 <small class="author" itemprop="author">Marilyn Monroe</small>,
 <small class="author" itemprop="author">Albert Einstein</small>,
 <small class="author" itemprop="author">André Gide</small>,
 <small class="author" itemprop="author">Thomas A. Edison</small>,
 <small class="author" itemprop="author">Eleanor Roosevelt</small>,
 <small class="author" itemprop="author">Steve Martin</small>]

2. Write a function to get **all** the authors and href links for the authors from the [homepage](http://quotes.toscrape.com/)


In [228]:
def authors(url):
    '''
    input: url
    
    return: a dictionary of authors and their urls
            {'author_1':'url_of_author_1', 'author_2':'url_of_author_2' ...}
    '''
    authors = {}
    html_page = requests.get(url) 
    soup = BeautifulSoup(html_page.content, 'html.parser')
    auths = soup.find_all(class_ = 'author')
    for a in auths:
        authors[a.text] = a.next_sibling.next_sibling['href']
    return authors

In [229]:
# run this cell to test the function
print(authors('http://quotes.toscrape.com/'))
print('\n')
print(authors('http://quotes.toscrape.com/page/3'))

{'Albert Einstein': '/author/Albert-Einstein', 'J.K. Rowling': '/author/J-K-Rowling', 'Jane Austen': '/author/Jane-Austen', 'Marilyn Monroe': '/author/Marilyn-Monroe', 'André Gide': '/author/Andre-Gide', 'Thomas A. Edison': '/author/Thomas-A-Edison', 'Eleanor Roosevelt': '/author/Eleanor-Roosevelt', 'Steve Martin': '/author/Steve-Martin'}


{'Pablo Neruda': '/author/Pablo-Neruda', 'Ralph Waldo Emerson': '/author/Ralph-Waldo-Emerson', 'Mother Teresa': '/author/Mother-Teresa', 'Garrison Keillor': '/author/Garrison-Keillor', 'Jim Henson': '/author/Jim-Henson', 'Dr. Seuss': '/author/Dr-Seuss', 'Albert Einstein': '/author/Albert-Einstein', 'J.K. Rowling': '/author/J-K-Rowling', 'Bob Marley': '/author/Bob-Marley'}


### Pagination

3. Get the first author on each of the first 5 pages of quotes. You can get to the next page with the next button at the bottom of the homepage.


In [231]:
# Your code here
peeps = []
for i in range(1,6):
    html_page = requests.get(f'http://quotes.toscrape.com/page/{i}') 
    soup = BeautifulSoup(html_page.content, 'html.parser')
    peeps.append(soup.find(class_='author').text)

In [232]:
peeps

['Albert Einstein',
 'Marilyn Monroe',
 'Pablo Neruda',
 'Dr. Seuss',
 'George R.R. Martin']

4. Write a function to get all of the quotes from a page.

In [305]:
bool(None)

False

In [314]:
def get_some_quotes(url = 'http://quotes.toscrape.com/', pages = None):
    '''
    input: url, number of pages to scrape (just scrape the home page if no argument is passed in)
    
    return: a list of dictionaries of quotes with their attributes
            [{'quote':'quote_1_text', 'author':'url_of_author_1'}, 
            {'quote':'quote_2_text', 'author':'url_of_author_2', 'quote_tags':[list_of_quote_2_tags]}, ...]
    '''
    quotes = []
    if pages:
        for i in range(1,pages+1):
            html_page = requests.get(f'http://quotes.toscrape.com/page/{i}') 
            soup = BeautifulSoup(html_page.content, 'html.parser')
            stuff = soup.find_all(class_='quote')
            for s in stuff:
                quotes.append({'quote': s.find(class_='text').text,
                               'author': s.find(class_='author').text,
                               'quote_tags': s.find('meta')['content'].split(',')})
    else:
        html_page = requests.get(url) 
        soup = BeautifulSoup(html_page.content, 'html.parser')    
        stuff = soup.find_all(class_='quote')
        for s in stuff:
            quotes.append({'quote': s.find(class_='text').text,
                           'author': s.find(class_='author').text,
                           'quote_tags': s.find('meta')['content'].split(',')})
    return quotes

In [315]:
soup.find_all(class_='quote')[0].find(class_='text')

<span class="text" itemprop="text">“A reader lives a thousand lives before he dies, said Jojen. The man who never reads lives only one.”</span>

In [318]:
# set the function to a variable to use later
quotes_for_mongo = get_some_quotes()
quotes_for_mongo

[{'quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'author': 'Albert Einstein',
  'quote_tags': ['change', 'deep-thoughts', 'thinking', 'world']},
 {'quote': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
  'author': 'J.K. Rowling',
  'quote_tags': ['abilities', 'choices']},
 {'quote': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'author': 'Albert Einstein',
  'quote_tags': ['inspirational', 'life', 'live', 'miracle', 'miracles']},
 {'quote': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
  'author': 'Jane Austen',
  'quote_tags': ['aliteracy', 'books', 'classic', 'humor']},
 {'quote': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
  'author': 'Mar

## Part 5: MongoDB

To do this section, open a connection to a mongo database in the terminal, using `mongod`. You will then **create**, **update**, and **read** from a mongo database.

Create and connect to a mongo database.

In [ ]:
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
mydb = myclient['quote_database']

In [ ]:
mycollection = mydb['quote_collection']

1. Add the quotes you obtained from the `get_some_quotes` function for the [homepage](http://quotes.toscrape.com/) to the mongo database. (You can also use the JSON file `quotes.json` to insert data into the database) To verify that you've successfully inserted the data, query it to obtain the resulting _ids back from the `results` variable. 

In [ ]:
# if you were unable to get the data from webscraping in the right format,
# uncomment the code below to access a JSON file with the list of dictionaries

# with open(r"data/quotes.json", "r") as r:
#     data = json.load(r)

In [ ]:
# use the results variable to confirm the data was inserted
results = None

2. Query the database for all the quotes written by `'Albert Einstein'`.

In [ ]:
q1 = None

3. Update Steve Martin's quote with the tags for the quote stored in the variable `steve_martin_tags`.

In [ ]:
steve_martin_tags = {'quote_tags': ['change', 'deep-thoughts', 'thinking', 'world']}
update_steve = None
first_quote_tags = None


4. Query the database to confirm that  `'Steve Martin'` is updated with `steve_martin_tags`.

In [ ]:
q2 = None